In [117]:
import numpy as np

def log_si_probs(num_tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission):
    si_probabilities = np.full((num_tags, len(word_indices)-1, num_tags), -np.inf)

    for i in range(len(word_indices)-1):
        for j in range(num_tags):
            for k in range(num_tags):
                si_probabilities[j, i, k] = (
                    log_forward[j, i] +
                    log_backward[k, i+1] +
                    transition[j, k] +
                    emission[k, word_indices[i+1]] -
                    log_forward_val
                )
    return si_probabilities



def log_si_probs_vec(log_forward, log_backward, log_forward_val, transition, log_emission_sentence):
    """
    Vectorized implementation of log_si_probs function.
    
    Parameters:
    - tags: List of possible tags
    - word_indices: Sequence of word indices
    - log_forward: Log forward probabilities
    - log_backward: Log backward probabilities
    - log_forward_val: Log forward value
    - transition: Transition log probabilities matrix
    - emission: Emission log probabilities matrix
    
    Returns:
    - si_probabilities: Vectorized log probabilities tensor
    """
    # Determine dimensions
    # num_tags = len(tags)
    # num_words = len(word_indices) - 1
    
    # # Create the output tensor filled with -inf
    # si_probabilities = np.full((num_tags, num_words, num_tags), -np.inf)
    
    # Compute the log probabilities using broadcasting
    # This replaces the nested loops with vectorized operations
        # Compute the full tensor of log probabilities for this word position

    print(f"{log_forward=}")
    print(f"{log_forward.shape=}")

    print(f"{log_backward=}")
    print(f"{log_backward.shape=}")

    print(f"{transition=}")
    print(f"{transition.shape=}")

    print(f"{log_emission_sentence=}")
    print(f"{log_emission_sentence.shape=}")

    print(f"{log_forward_val=}")
    return (
        log_forward[:, :-1, np.newaxis] +  # first tag dimension
        log_backward[:, 1:].T[np.newaxis, :] +  # second tag dimension
        transition[:, np.newaxis, :] +  # transition probabilities
        log_emission_sentence[:, 1:].T[np.newaxis, :] -  # emission for next word
        log_forward_val
    )
    
    return si_probabilities




In [72]:
import numpy as np

# Define the parameters
tags = ['Noun', 'Verb', 'Adjective']
word_sequence = ['the', 'quick', 'brown', 'fox']
word_indices = [1, 3, 2, 0]  # corresponding indices in a hypothetical vocabulary
log_forward = np.array([
    [1, 2, 3, 4],  # Values for Noun
    [5, 6, 7, 8],  # Values for Verb
    [9, 10, 11, 12]  # Values for Adjective
])
log_backward = np.array([
    [120, 110, 100, 90],  # Values for Noun
    [80, 70, 60, 50],  # Values for Verb
    [40, 30, 20, 10]  # Values for Adjective
])
log_forward_val = 10

# Define transition matrix with distinct values
transition = np.array([
    [1, 2, 3],  # From Noun to Noun, Verb, Adjective
    [4, 5, 6],  # From Verb to Noun, Verb, Adjective
    [7, 8, 9]   # From Adjective to Noun, Verb, Adjective
])
# Define emission matrix with distinct values
vocab_size = max(word_indices) + 1
emission = np.array([
    [11, 22, 33, 44],  # Emissions for Noun
    [88, 77, 66, 55],  # Emissions for Verb
    [99, 111, 222, 333]   # Emissions for Adjective
])

# Print input values
print("Tags:", tags)
print("Word Indices:", word_indices)
print("Word Sequence:", word_sequence)
print("Forward Matrix:\n", log_forward)
print("Backward Matrix:\n", log_backward)
print("Forward Value:", log_forward_val)
print("Transition Matrix:\n", transition)
print("Emission Matrix:\n", emission)

# Use the original function
SI_probabilities = log_si_probs(tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission)
print("SI Probabilities from log_si_probs:\n", SI_probabilities)

# Use the vectorized function
SI_probabilities_vec = log_si_probs_vec(tags, word_indices, log_forward, log_backward, log_forward_val, transition, emission)
print("SI Probabilities from log_si_probs_vec:\n", SI_probabilities_vec)

# Check if the outputs are equivalent
print("Are the outputs equivalent?", np.allclose(SI_probabilities, SI_probabilities_vec))


Tags: ['Noun', 'Verb', 'Adjective']
Word Indices: [1, 3, 2, 0]
Word Sequence: ['the', 'quick', 'brown', 'fox']
Forward Matrix:
 [[ 1  2  3  4]
 [ 5  6  7  8]
 [ 9 10 11 12]]
Backward Matrix:
 [[120 110 100  90]
 [ 80  70  60  50]
 [ 40  30  20  10]]
Forward Value: 10
Transition Matrix:
 [[1 2 3]
 [4 5 6]
 [7 8 9]]
Emission Matrix:
 [[ 11  22  33  44]
 [ 88  77  66  55]
 [ 99 111 222 333]]
SI Probabilities from log_si_probs:
 [[[146. 118. 357.]
  [126. 120. 237.]
  [ 95. 133. 105.]]

 [[153. 125. 364.]
  [133. 127. 244.]
  [102. 140. 112.]]

 [[160. 132. 371.]
  [140. 134. 251.]
  [109. 147. 119.]]]
SI Probabilities from log_si_probs_vec:
 [[[146 118 357]
  [126 120 237]
  [ 95 133 105]]

 [[153 125 364]
  [133 127 244]
  [102 140 112]]

 [[160 132 371]
  [140 134 251]
  [109 147 119]]]
Are the outputs equivalent? True


In [1]:
import numpy as np

# Parameters
num_tags = 3
num_words = 16
word_indices = np.array([i for i in range(16)])  # Example word indices
log_forward = np.random.rand(num_tags, num_words)  # Shape: (num_tags, num_words)
log_backward = np.random.rand(num_tags, num_words)  # Shape: (num_tags, num_words)
log_transition = np.random.rand(num_tags, num_tags)  # Shape: (num_tags, num_tags)
log_emission = np.random.rand(num_tags, num_words)  # Shape: (num_tags, num_words)
log_forward_val = np.random.rand()  # Scalar for simplicity

# Prepare for vectorized computation
next_word_indices = word_indices[1:]  # Shape: (num_words - 1,)

log_emission_sentence = log_emission[:, word_indices]

# Expand dimensions to align shapes for broadcasting
log_forward_expanded = log_forward[:, :-1, np.newaxis]
log_backward_expanded = log_backward[:, 1:].T[np.newaxis, :]
log_transition_expanded = log_transition[:, np.newaxis, :]
log_emission_expanded = log_emission_sentence[:, 1:].T[np.newaxis, :]

# Print results to verify shapes
print("log_forward shape:", log_forward.shape)
print("log_backward shape:", log_backward.shape)
print("log_transition shape:", log_transition.shape)
print("log_emission shape:", log_emission.shape)
print("log_forward_expanded shape:", log_forward_expanded.shape)
print("log_backward_expanded shape:", log_backward_expanded.shape)
print("log_transition_expanded shape:", log_transition_expanded.shape)
print("log_emission_expanded shape:", log_emission_expanded.shape)

# Fully vectorized computation
si_probs = (
    log_forward_expanded +
    log_backward_expanded +
    log_transition_expanded +
    log_emission_expanded -
    log_forward_val
)  # Final shape: (num_tags, num_words-1, num_tags)
print("si_probs shape:", si_probs.shape)


print(si_probs)


log_forward shape: (3, 16)
log_backward shape: (3, 16)
log_transition shape: (3, 3)
log_emission shape: (3, 16)
log_forward_expanded shape: (3, 15, 1)
log_backward_expanded shape: (1, 15, 3)
log_transition_expanded shape: (3, 1, 3)
log_emission_expanded shape: (1, 15, 3)
si_probs shape: (3, 15, 3)
[[[0.66114421 1.22219175 2.22468667]
  [2.08331742 2.1094548  1.52780613]
  [1.29384782 2.49302671 2.22043913]
  [2.32852928 2.26259848 2.07784851]
  [1.53940877 0.89471254 0.54066099]
  [0.98628689 1.5304961  1.51890593]
  [1.54700947 0.79884621 1.30599582]
  [2.20457659 1.76904227 1.88809076]
  [2.06422403 1.63954274 1.71387017]
  [2.0287846  1.66068971 2.4593483 ]
  [2.29036038 1.47356807 1.57253262]
  [1.97639134 2.04910547 2.66755613]
  [1.69628967 2.38655442 2.932019  ]
  [1.74266376 2.89546503 2.8749465 ]
  [1.27737396 1.81551306 1.78008485]]

 [[0.37570308 0.92981094 1.87278586]
  [1.87487211 1.89406982 1.25290114]
  [1.37188807 2.56412729 2.2320197 ]
  [1.6792594  1.60638893 1.362118

In [73]:
import numpy as np
from scipy.special import logsumexp
import scipy.sparse as sc

def compute_a_loop(num_tags, word_indices, log_si_probabilities):
    log_si_probabilities = np.exp(log_si_probabilities)
    a = np.zeros((num_tags, num_tags))
    for j in range(num_tags):
        for i in range(num_tags):
            for t in range(len(word_indices)-1):
                a[j,i] = a[j,i] + log_si_probabilities[j,t,i]

            denomenator_a = [log_si_probabilities[j, t_x, i_x] for t_x in range(len(word_indices) - 1) for i_x in range(num_tags)]
            denomenator_a = sum(denomenator_a)

            if (denomenator_a == 0):
                a[j,i] = 0
            else:
                a[j,i] = a[j,i]/denomenator_a
    return np.log(a)

def compute_a_vectorized(num_tags, word_indices, log_si_probabilities):

    # # Summing over time steps for each transition pair (j, i)
    # a_log = logsumexp(log_si_probabilities, axis=1)  # Shape: (num_tags, num_tags)

    # # Normalizing across each row
    # denom_a_log = logsumexp(log_si_probabilities, axis=(1, 2))[:, np.newaxis]  # Shape: (num_tags, 1)
    # # print(log_si_probabilities.shape)
    # # print(a_log.shape)
    # # print(denom_a_log.shape)
    # a_log_normalized = a_log - denom_a_log  # Broadcasting to normalize each row
    a_log_normalized = logsumexp(log_si_probabilities, axis=1) - logsumexp(log_si_probabilities, axis=(1, 2))[:, np.newaxis]

    return a_log_normalized

def compute_b_loop(num_tags, words_dict, word_indices, log_gamma_probabilities):
    b_log = np.full((num_tags, len(words_dict)), -np.inf)

    for j in range(num_tags):  # Loop over states
        for i in range(len(words_dict)):  # Loop over observation symbols
            # Find indices where the observation matches the current symbol
            indices = [idx for idx, val in enumerate(word_indices) if val == words_dict[i]]
            
            if indices:  # Only process if there are matching indices
                numerator_logs = log_gamma_probabilities[j, indices]
                # print(f"{j=}, {i=}, {indices=}, {numerator_logs=}")
                numerator_b_log = logsumexp(numerator_logs)
            else:
                numerator_b_log = -np.inf
            denomenator_logs = log_gamma_probabilities[j, :]
            denomenator_b_log = logsumexp(denomenator_logs)
            # print(f"{j=}, {i=}, {numerator_b_log=}, {denomenator_b_log=}")


            # Avoid log(0) issues by checking the denominator
            if np.isneginf(denomenator_b_log):
                b_log[j, i] = -np.inf  # Probability is 0
            else:
                b_log[j, i] = numerator_b_log - denomenator_b_log
    return b_log

def compute_b_vectorized(num_tags, num_words, word_indices, log_gamma_probabilities):
    # Assume `word_indices`, `log_gamma_probabilities`, and `states` are already defined

    b_denominator_logs = logsumexp(log_gamma_probabilities, axis=1, keepdims=True)  # Shape: (num_states, 1)
    # Pray we don't need to handle cases where denominator is -inf
    assert np.all(~np.isneginf(b_denominator_logs)), "Error: denominator contains -inf values."
    unique_word_indices, inverse_indices = np.unique(word_indices, return_inverse=True)

    # sparse_mask = sc.coo_matrix(
    # (np.ones_like(inverse_indices), (inverse_indices, np.arange(len(inverse_indices)))),
    # shape=(len(unique_word_indices), len(inverse_indices))
    # )

    # # Compute the log-sum-exp in a vectorized way
    # numerator_logs = np.full((log_gamma_probabilities.shape[0], len(unique_word_indices)), -np.inf)  # Initialize with log(0)
    # for state in range(log_gamma_probabilities.shape[0]):  # Iterate over states
    #     # Multiply sparse mask with the log_gamma_probabilities for this state
    #     numerator_logs[state, :] = logsumexp(
    #         log_gamma_probabilities[state, :] + sparse_mask.toarray(), axis=1
    #     )

    num_unique_words = len(unique_word_indices)
    b_numerator_logs = np.full((num_tags, num_unique_words), -np.inf)
    # np.add.at(numerator_logs, (slice(None), inverse_indices), log_gamma_probabilities)
    for idx in range(num_unique_words):
        mask = (inverse_indices == idx)
        b_numerator_logs[:, idx] = logsumexp(log_gamma_probabilities[:, mask], axis=1)

    log_b = np.full((num_tags, num_words), -np.inf)
    log_b[:, unique_word_indices] = b_numerator_logs - b_denominator_logs
    
    return log_b



In [74]:
# Dummy variables
num_tags = 7
word_indices = [1, 2, 3, 1]
words_dict = {0: 0, 1: 1, 2: 2, 3:3, 4:4, 5:5}
log_si_probabilities = np.random.uniform(-10, 0, (num_tags, len(word_indices) - 1, num_tags))
log_gamma_probabilities = np.random.uniform(-10, 0, (num_tags, len(word_indices)))

# Compute "a" matrices
log_a_loop = compute_a_loop(num_tags, word_indices, log_si_probabilities)
log_a_vectorized = compute_a_vectorized(num_tags, word_indices, log_si_probabilities)

# Compute "b" matrices
log_b_loop = compute_b_loop(num_tags, words_dict, word_indices, log_gamma_probabilities)
log_b_vectorized = compute_b_vectorized(num_tags, len(words_dict), word_indices, log_gamma_probabilities)

# Print and verify outputs
print("log_a_loop:\n", log_a_loop)
print("log_a_vectorized:\n", log_a_vectorized)
print("Are 'a' matrices equal?", np.allclose(log_a_loop, log_a_vectorized, atol=1e-6))

print("\nlog_b_loop:\n", log_b_loop)
print("log_b_vectorized:\n", log_b_vectorized)
print("Are 'b' matrices equal?", np.allclose(log_b_loop, log_b_vectorized, atol=1e-6))


log_a_loop:
 [[-1.10219086 -3.66166884 -1.61652908 -3.74922431 -1.3043017  -2.08406434
  -3.7189444 ]
 [-2.64303872 -2.19181939 -1.60251642 -4.43653522 -6.6008387  -0.57463624
  -3.22773675]
 [-1.85253657 -1.36519206 -3.03809853 -2.65899614 -0.86333553 -3.06796228
  -6.42760087]
 [-1.44441396 -1.86768374 -1.25691842 -6.71845405 -1.93542335 -5.42602742
  -1.74220905]
 [-5.21784892 -1.24818718 -2.27448585 -2.24581097 -3.56983191 -2.91484588
  -0.87588812]
 [-4.12268407 -3.49049773 -0.5819955  -1.35982249 -6.93000815 -3.80409604
  -2.16659137]
 [-3.19633269 -2.93555601 -5.09372299 -0.43832788 -1.76650251 -3.00104091
  -3.37926641]]
log_a_vectorized:
 [[-1.10219086 -3.66166884 -1.61652908 -3.74922431 -1.3043017  -2.08406434
  -3.7189444 ]
 [-2.64303872 -2.19181939 -1.60251642 -4.43653522 -6.6008387  -0.57463624
  -3.22773675]
 [-1.85253657 -1.36519206 -3.03809853 -2.65899614 -0.86333553 -3.06796228
  -6.42760087]
 [-1.44441396 -1.86768374 -1.25691842 -6.71845405 -1.93542335 -5.42602742
  -

In [2]:
import numpy as np


#generating initial probabilities

#transition probabilities
transition = np.array([[0.8,0.1],
                       [0.1,0.8]])
#Emission probabilities
emission = np.array([[0.1,0.2,0.7],
                     [0.7,0.2,0.1]])

#defining states and sequence symbols
states = ['H','C']
states_dic = {'H':0, 'C':1}
sequence_syms = {'1':0,'2':1,'3':2}
sequence = ['1','2','3']

#test sequence
test_sequence = '331122313'
test_sequence = [x for x in test_sequence]

#probabilities of going to end state
end_probs = [0.1, 0.1]
#probabilities of going from start state
start_probs = [0.5, 0.5]


#function to find forward probabilities
def forward_probs():
    # node values stored during forward algorithm
    node_values_fwd = np.zeros((len(states), len(test_sequence)))

    for i, sequence_val in enumerate(test_sequence):
        for j in range(len(states)):
            # if first sequence value then do this
            if (i == 0):
                node_values_fwd[j, i] = start_probs[j] * emission[j, sequence_syms[sequence_val]]
            # else perform this
            else:
                values = [node_values_fwd[k, i - 1] * emission[j, sequence_syms[sequence_val]] * transition[k, j] for k in
                          range(len(states))]
                node_values_fwd[j, i] = sum(values)

    #end state value
    end_state = np.multiply(node_values_fwd[:, -1], end_probs)
    end_state_val = sum(end_state)
    return node_values_fwd, end_state_val



#function to find backward probabilities
def backward_probs():
    # node values stored during forward algorithm
    node_values_bwd = np.zeros((len(states), len(test_sequence)))

    #for i, sequence_val in enumerate(test_sequence):
    for i in range(1,len(test_sequence)+1):
        for j in range(len(states)):
            # if first sequence value then do this
            if (-i == -1):
                node_values_bwd[j, -i] = end_probs[j]
            # else perform this
            else:
                values = [node_values_bwd[k, -i+1] * emission[k, sequence_syms[test_sequence[-i+1]]] * transition[j, k] for k in range(len(states))]
                node_values_bwd[j, -i] = sum(values)

    #start state value
    start_state = [node_values_bwd[m,0] * emission[m, sequence_syms[test_sequence[0]]] for m in range(len(states))]
    start_state = np.multiply(start_state, start_probs)
    start_state_val = sum(start_state)
    return node_values_bwd, start_state_val


#function to find si probabilities
def si_probs(forward, backward, forward_val):

    si_probabilities = np.zeros((len(states), len(test_sequence)-1, len(states)))

    for i in range(len(test_sequence)-1):
        for j in range(len(states)):
            for k in range(len(states)):
                si_probabilities[j,i,k] = ( forward[j,i] * backward[k,i+1] * transition[j,k] * emission[k,sequence_syms[test_sequence[i+1]]] ) \
                                                    / forward_val
    return si_probabilities

#function to find gamma probabilities
def gamma_probs(forward, backward, forward_val):

    gamma_probabilities = np.zeros((len(states), len(test_sequence)))

    for i in range(len(test_sequence)):
        for j in range(len(states)):
            #gamma_probabilities[j,i] = ( forward[j,i] * backward[j,i] * emission[j,sequence_syms[test_sequence[i]]] ) / forward_val
            gamma_probabilities[j, i] = (forward[j, i] * backward[j, i]) / forward_val

    return gamma_probabilities



#performing iterations until convergence

for iteration in range(2000):

    print('\nIteration No: ', iteration + 1)
    # print('\nTransition:\n ', transition)
    # print('\nEmission: \n', emission)

    #Calling probability functions to calculate all probabilities
    fwd_probs, fwd_val = forward_probs()
    bwd_probs, bwd_val = backward_probs()
    si_probabilities = si_probs(fwd_probs, bwd_probs, fwd_val)
    gamma_probabilities = gamma_probs(fwd_probs, bwd_probs, fwd_val)

    # print('Forward Probs:')
    # print(np.matrix(fwd_probs))
    #
    # print('Backward Probs:')
    # print(np.matrix(bwd_probs))
    #
    # print('Si Probs:')
    # print(si_probabilities)
    #
    # print('Gamma Probs:')
    # print(np.matrix(gamma_probabilities))

    #caclculating 'a' and 'b' matrices
    a = np.zeros((len(states), len(states)))
    b = np.zeros((len(states), len(sequence_syms)))

    #'a' matrix
    for j in range(len(states)):
        for i in range(len(states)):
            for t in range(len(test_sequence)-1):
                a[j,i] = a[j,i] + si_probabilities[j,t,i]

            denomenator_a = [si_probabilities[j, t_x, i_x] for t_x in range(len(test_sequence) - 1) for i_x in range(len(states))]
            denomenator_a = sum(denomenator_a)

            if (denomenator_a == 0):
                a[j,i] = 0
            else:
                a[j,i] = a[j,i]/denomenator_a

    #'b' matrix
    for j in range(len(states)): #states
        for i in range(len(sequence)): #seq
            indices = [idx for idx, val in enumerate(test_sequence) if val == sequence[i]]
            numerator_b = sum( gamma_probabilities[j,indices] )
            denomenator_b = sum( gamma_probabilities[j,:] )

            if (denomenator_b == 0):
                b[j,i] = 0
            else:
                b[j, i] = numerator_b / denomenator_b


    print('\nMatrix a:\n')
    print(np.matrix(a.round(decimals=4)))
    print('\nMatrix b:\n')
    print(np.matrix(b.round(decimals=4)))

    transition = a
    emission = b

    new_fwd_temp, new_fwd_temp_val = forward_probs()
    print('New forward probability: ', new_fwd_temp_val)
    diff =  np.abs(fwd_val - new_fwd_temp_val)
    print('Difference in forward probability: ', diff)

    if (diff < 0.0000001):
        break


c = 1


Iteration No:  1

Matrix a:

[[0.7859 0.2141]
 [0.2094 0.7906]]

Matrix b:

[[0.2005 0.1735 0.626 ]
 [0.5016 0.284  0.2145]]
New forward probability:  5.695152119409513e-06
Difference in forward probability:  5.002073262273513e-06

Iteration No:  2

Matrix a:

[[0.7185 0.2815]
 [0.2161 0.7839]]

Matrix b:

[[0.235  0.1419 0.6231]
 [0.4256 0.2976 0.2768]]
New forward probability:  7.128566815415427e-06
Difference in forward probability:  1.4334146960059136e-06

Iteration No:  3

Matrix a:

[[0.6595 0.3405]
 [0.2027 0.7973]]

Matrix b:

[[0.2536 0.1131 0.6333]
 [0.3925 0.3032 0.3043]]
New forward probability:  7.898268000620344e-06
Difference in forward probability:  7.697011852049176e-07

Iteration No:  4

Matrix a:

[[0.6083 0.3917]
 [0.1856 0.8144]]

Matrix b:

[[0.2609 0.0838 0.6553]
 [0.378  0.3076 0.3144]]
New forward probability:  8.505915984403414e-06
Difference in forward probability:  6.076479837830701e-07

Iteration No:  5

Matrix a:

[[0.5653 0.4347]
 [0.1681 0.8319]]

Matri

In [112]:
def forward_log_vec(num_tags, num_words, log_initial, log_transition, log_emission_sentence):
    log_probs = np.full((num_tags, num_words), -np.inf)
    scaling_factors = np.zeros(num_words)
    
    # Initial step with normalization
    log_probs[:, 0] = log_initial + log_emission_sentence[:, 0]
    factor0 = logsumexp(log_probs[:, 0])
    log_probs[:, 0] -= factor0
    scaling_factors[0] = factor0
    
    for t in range(1, num_words):
        # Compute log probabilities
        log_probs[:, t] = logsumexp(log_probs[:, t-1][:, np.newaxis] + log_transition, axis=0) + log_emission_sentence[:, t]
        
        # Normalize at each time step
        factor = logsumexp(log_probs[:, t])
        log_probs[:, t] -= factor
        scaling_factors[t] = factor
    print(scaling_factors)
    
    return log_probs, logsumexp(scaling_factors)

def backward_log_vec(num_tags, num_words, log_transition, log_emission_sentence):
    # num_words = len(word_indices)
    backward_vals = np.full((num_tags, num_words), -np.inf)
    # Last column, deduct scaling factor
    backward_vals[:, -1] = 0
    backward_vals[:, -1] -= logsumexp(backward_vals[:, -1])
    for t in range(num_words-2, -1, -1):
        backward_vals[:, t] = logsumexp(backward_vals[:, t + 1][:, np.newaxis] + log_transition, axis=1) + log_emission_sentence[:, t + 1]
        # NORMALISE
        backward_vals[:, t] -= logsumexp(backward_vals[:, t])
    print(f"{backward_vals=}")

    return backward_vals

num_tags = 3
num_words = 4
log_initial = np.log(np.array([0.2, 0.3, 0.5]))
log_transition = np.log(np.array([[0.1, 0.6, 0.3],
                                  [0.4, 0.1, 0.5],
                                  [0.3, 0.5, 0.2]]))
log_emission_sentence = np.log(np.array([[0.5, 0.1, 0.4, 0.7],
                                         [0.2, 0.8, 0.1, 0.3],
                                         [0.3, 0.1, 0.5, 0.2]]))

# Call forward and backward functions
log_probs_forward, forward_value = forward_log_vec(num_tags, num_words, log_initial, log_transition, log_emission_sentence)
log_probs_backward = backward_log_vec(num_tags, num_words, log_transition, log_emission_sentence)

# Assertion
# assert np.isclose(forward_scale_sum, backward_scale_sum), "The sums of scaling factors should be the same"

print("Forward:", log_probs_forward)
print("Backward:", log_probs_backward)
print(forward_value)
# print("Assertion passed: Forward and backward scaling factors are equal.")

[-1.17118298 -0.87134821 -0.9182008  -1.00348155]
backward_vals=array([[-0.79492987, -0.38136756, -0.5389965 , -1.09861229],
       [-0.94908055, -2.61495978, -1.38629436, -1.09861229],
       [-1.82454929, -1.41098697, -1.79175947, -1.09861229]])
Forward: [[-1.13140211 -2.79836133 -0.97948475 -0.82782974]
 [-1.64222774 -0.13960775 -3.22820836 -0.85115559]
 [-0.725937   -2.66799951 -0.53637075 -1.99449731]]
Backward: [[-0.79492987 -0.38136756 -0.5389965  -1.09861229]
 [-0.94908055 -2.61495978 -1.38629436 -1.09861229]
 [-1.82454929 -1.41098697 -1.79175947 -1.09861229]]
0.40160745935373554


In [118]:
# Print input values
# print("Tags:", tags)
# print("Word Indices:", word_indices)
# print("Word Sequence:", word_sequence)
print("Forward Matrix:\n", log_probs_backward)
print("Backward Matrix:\n", log_probs_backward)
print("Forward Value:", forward_value)
print("Transition Matrix:\n", log_transition)
print("Emission Matrix:\n", log_emission_sentence)

word_indicess = [0, 1, 2, 3]

# Use the original function
SI_probabilities = log_si_probs(num_tags, word_indicess, log_probs_forward, log_probs_backward, forward_value, log_transition, log_emission_sentence)
print("SI Probabilities from log_si_probs:\n", SI_probabilities)

# Use the vectorized function
SI_probabilities_vec = log_si_probs_vec(log_probs_forward, log_probs_backward, forward_value, log_transition, log_emission_sentence)
print("SI Probabilities from log_si_probs_vec:\n", SI_probabilities_vec)

# Check if the outputs are equivalent
print("Are the outputs equivalent?", np.allclose(SI_probabilities, SI_probabilities_vec))

Forward Matrix:
 [[-0.79492987 -0.38136756 -0.5389965  -1.09861229]
 [-0.94908055 -2.61495978 -1.38629436 -1.09861229]
 [-1.82454929 -1.41098697 -1.79175947 -1.09861229]]
Backward Matrix:
 [[-0.79492987 -0.38136756 -0.5389965  -1.09861229]
 [-0.94908055 -2.61495978 -1.38629436 -1.09861229]
 [-1.82454929 -1.41098697 -1.79175947 -1.09861229]]
Forward Value: 0.40160745935373554
Transition Matrix:
 [[-2.30258509 -0.51082562 -1.2039728 ]
 [-0.91629073 -2.30258509 -0.69314718]
 [-1.2039728  -0.69314718 -1.60943791]]
Emission Matrix:
 [[-0.69314718 -2.30258509 -0.91629073 -0.35667494]
 [-1.60943791 -0.22314355 -2.30258509 -1.2039728 ]
 [-1.2039728  -2.30258509 -0.69314718 -1.60943791]]


ValueError: setting an array element with a sequence.